#### Normaliza o texto [transcrições]

In [ ]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_languange, get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','transcribes','transcription_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    normal = normalize_text(get_data('videos_data.db','transcribes','transcription',id)[0][1],get_languange('videos_data.db','links',id)[0])
    update_data('videos_data.db','transcribes','transcription_normalizado',id,normal)


#### Normaliza o texto [captions]

In [ ]:
from Funcs.tratamentoDeTexto import normalize_text
from Funcs.getPaths import get_Paths
from Funcs.db import get_data, update_data

from Funcs.db import new_collumn
new_collumn('videos_data.db','captions','captions_normalizado','TEXT')

paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]
    frames = get_data('videos_data.db','captions','caption',id)
    for frame in frames:
        #print('id: '+frame[0])
        #print('caption: '+frame[1])
        normal = normalize_text(frame[1],'en')
        update_data('videos_data.db','captions','captions_normalizado',frame[0],normal)
    






#### Vetorizar [captions]

In [ ]:
# Adicionar todas as strings em uma única variável

from Funcs.db import get_data, push_data
from Funcs.getPaths import get_Paths
from sklearn.feature_extraction.text import TfidfVectorizer
import json

#all_captionsNids = []
ids = []
textos = []
paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]   # Videos\\<id>
    
    frames_with_id = get_data('videos_data.db','captions','captions_normalizado',id)    # Retorna um conjunto de frames do video <id> o id do frame + a caption normalizada 
    aux_ids, aux_captions = zip(*frames_with_id)

    ids.extend(aux_ids)
    textos.extend(aux_captions)


vectorizer = TfidfVectorizer()
vetores = vectorizer.fit_transform(textos)

data_to_save = []
for idx, id_ in enumerate(ids):
    vector_data = vetores[idx].tocoo()

    vector_elements = [
        ((int(idx), int(col)), float(data))
        for (row, col), data in zip(zip(vector_data.row, vector_data.col), vector_data.data)
    ]
    data_to_save.append({
        'id': id_,
        'vector': vector_elements
    })

for item in data_to_save:
    video_id = item['id'].split("_")[0]
    vector_json = json.dumps(item['vector'])
    push_data('videos_data.db', 'textos_vetorizados', 'vetor', item['id'], vector_json, video_id)

#### Vetorizar [transcrições]

In [5]:
# Adicionar todas as strings em uma única variável

from Funcs.db import get_data, push_data
from Funcs.getPaths import get_Paths
from sklearn.feature_extraction.text import TfidfVectorizer
import json


#all_captionsNids = []
ids = []
textos = []
paths = get_Paths('Videos')
for path in paths:
    id = path.split("\\")[-1]   # Videos\\<id>
    
    trancript_with_id = get_data('videos_data.db','transcribes','transcription_normalizado',id)    # Retorna um conjunto de transcrições com id
    aux_ids, aux_transcript = zip(*trancript_with_id)

    ids.append(aux_ids[0])
    textos.append(aux_transcript[0])

vectorizer = TfidfVectorizer()
vetores = vectorizer.fit_transform(textos)

data_to_save = []
for idx, id_ in enumerate(ids):
    vector_data = vetores[idx].tocoo()
    vector_elements = [
        ((int(idx), int(col)), float(data))
        for (row, col), data in zip(zip(vector_data.row, vector_data.col), vector_data.data)
    ]
    data_to_save.append({
        'id': id_,
        'vector': vector_elements
    })


for item in data_to_save:
    vector_json = json.dumps(item['vector'])
    #push_data('videos_data.db', 'transcriptions_vetorizados', 'vetor', item['id'], vector_json, item['id'])

    
   